# Instalando dependências

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [1]:
import gymnasium as gym
import torch
from util.algorithms import run_sarsa
from util.network import ReplayBuffer
from IPython.display import clear_output

In [2]:
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural

In [3]:
# Passo 1: Coletar um conjunto fixo de transições (Replay Buffer)
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]
ENVS_REPLAY_BUFFER = []
for i, env_name in enumerate(ENV_NAMES):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    env = gym.make(env_name, render_mode="rgb_array")
    
    sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, sum_rewards_per_ep, q))
    replay_buffer.save_config(f"config\dataset\sarsa\{env_name}.json")


Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>
OI
Episódio 0 terminou com recompensa 0.0 na transição 48
Episódio 100 terminou com recompensa 0.0 na transição 1574
Episódio 200 terminou com recompensa 0.0 na transição 3099
Episódio 300 terminou com recompensa 0.0 na transição 4639
Episódio 400 terminou com recompensa 0.0 na transição 6208
Episódio 500 terminou com recompensa 0.0 na transição 7626
Episódio 600 terminou com recompensa 0.0 na transição 9061
Episódio 700 terminou com recompensa 0.0 na transição 10896
Episódio 800 terminou com recompensa 0.0 na transição 12441
Episódio 900 terminou com recompensa 0.0 na transição 13882
Episódio 1000 terminou com recompensa 0.0 na transição 15278
Episódio 1100 terminou com recompensa 0.0 na transição 16510
Episódio 1200 terminou com recompensa 0.0 na transição 18038
Episódio 1300 terminou com recompensa 0.0 na transição 19473
Episódio 1400 terminou com recompensa 0.0 na transição 20808
Episódio 1500

# Treinando o Modelo

## Carregando Datasets

In [4]:
import gymnasium as gym
from util.network import ReplayBuffer


In [5]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]

In [6]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
    

<string>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<string>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\lucam\AppData\Local\Temp\ipykernel_5968\4138853788.py:3: SyntaxWarning: invalid escape sequence '\{'
  replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
C:\Users\lucam\AppData\Local\Temp\ipykernel_5968\4138853788.py:3: SyntaxWarning: invalid escape sequence '\d'
  replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")


## Definições de Treino

In [2]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]

In [7]:
import argparse

In [8]:
def get_config(env_name, count_episodes, seed, gamma, tau, alpha, lr, steps, hidden_size=256):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-FQI", help="Run name, default: CQL-DQN")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name, default: CartPole-v0")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes, default: 200")
    parser.add_argument("--seed", type=int, default=seed, help="Seed, default: 1")
    parser.add_argument("--steps", type=int, default=steps, help="Saves the network every x epochs, default: 25")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--tau", type=float, default=tau, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--alpha", type=float, default=alpha, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--lr", type=float, default=lr, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--hidden_size", type=int, default=hidden_size, help="Saves the network every x epochs, default: 25")
    
    args, _ = parser.parse_known_args()
    return args

In [9]:
import wandb
from collections import deque
from util.network import CQLAgent, save, to_one_hot
import numpy as np
import random
import torch

In [10]:
def train_DQN_FQI(config, buffer, model):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)
    env = gym.make(config.env)

    #env.seed(config.seed)
    #env.action_space.seed(config.seed)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    agent = CQLAgent(env.observation_space.n, env.action_space.n, config.tau, config.gamma, config.lr, config.alpha, model, config.hidden_size, device)

    for i in range(config.steps):
        bellmann_error = agent.FQIlearn(buffer.sample())
        agent.alpha = config.alpha * np.exp(-3e-6 * i)            
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Q Loss: {bellmann_error} ")
    
    save(config, save_name="FQI-DQN", model=agent.network, wandb=wandb, ep=i)
    
    return env

In [11]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.long)
            state_tensor = to_one_hot(state_tensor, model.input_size)[0].float().unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [12]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(state_tensor, q_network.input_size)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [13]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [14]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.input_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"fqi_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

## FrozenLake

In [48]:
from util.network import FrozenLakeNet

In [49]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 64
STEPS = 100_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = 128
# Extra:

In [50]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [51]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [52]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[0][2], FrozenLakeNet)

[TRAIN 0] Q Loss: 0.003491026349365711 
[TRAIN 1000] Q Loss: 0.0006482771714217961 
[TRAIN 2000] Q Loss: 0.003181132022291422 
[TRAIN 3000] Q Loss: 0.00401427922770381 
[TRAIN 4000] Q Loss: 0.004122424870729446 
[TRAIN 5000] Q Loss: 0.004780708812177181 
[TRAIN 6000] Q Loss: 0.006060981657356024 
[TRAIN 7000] Q Loss: 0.004536996595561504 
[TRAIN 8000] Q Loss: 0.005002215504646301 
[TRAIN 9000] Q Loss: 0.003729019546881318 
[TRAIN 10000] Q Loss: 0.007827803492546082 
[TRAIN 11000] Q Loss: 0.002547140698879957 
[TRAIN 12000] Q Loss: 0.00839303433895111 
[TRAIN 13000] Q Loss: 0.007079601753503084 
[TRAIN 14000] Q Loss: 0.005071827210485935 
[TRAIN 15000] Q Loss: 0.006898911669850349 
[TRAIN 16000] Q Loss: 0.0037578754127025604 
[TRAIN 17000] Q Loss: 0.003332689171656966 
[TRAIN 18000] Q Loss: 0.006630145013332367 
[TRAIN 19000] Q Loss: 0.003930478356778622 
[TRAIN 20000] Q Loss: 0.004539133980870247 
[TRAIN 21000] Q Loss: 0.003719568718224764 
[TRAIN 22000] Q Loss: 0.0036416449584066868 


In [53]:
import torch

In [54]:
env = ENVS_REPLAY_BUFFER[0][1]
model = FrozenLakeNet(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

FrozenLakeNet(
  (fc1): Linear(in_features=16, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)

In [55]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 0.76


In [56]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

## Taxi

In [15]:
from util.network import TaxiNet

In [60]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 300_000
GAMMA = 0.97  # Fator de desconto
TAU = 1e-2      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = 512
# Extra:

In [61]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [62]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [63]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[1][2], TaxiNet)

[TRAIN 0] Q Loss: 0.668156087398529 
[TRAIN 1000] Q Loss: 0.10242307186126709 
[TRAIN 2000] Q Loss: 0.3969406187534332 
[TRAIN 3000] Q Loss: 0.36752933263778687 
[TRAIN 4000] Q Loss: 0.45255741477012634 
[TRAIN 5000] Q Loss: 0.6982078552246094 
[TRAIN 6000] Q Loss: 0.45209425687789917 
[TRAIN 7000] Q Loss: 0.38023871183395386 
[TRAIN 8000] Q Loss: 0.4073435664176941 
[TRAIN 9000] Q Loss: 0.5858455896377563 
[TRAIN 10000] Q Loss: 0.4121379256248474 
[TRAIN 11000] Q Loss: 0.2946318984031677 
[TRAIN 12000] Q Loss: 0.8561798930168152 
[TRAIN 13000] Q Loss: 0.7895463705062866 
[TRAIN 14000] Q Loss: 0.25231796503067017 
[TRAIN 15000] Q Loss: 0.9841479063034058 
[TRAIN 16000] Q Loss: 0.8089545369148254 
[TRAIN 17000] Q Loss: 0.529068112373352 
[TRAIN 18000] Q Loss: 0.06028711795806885 
[TRAIN 19000] Q Loss: 0.05823388323187828 
[TRAIN 20000] Q Loss: 0.08793574571609497 
[TRAIN 21000] Q Loss: 0.13181382417678833 
[TRAIN 22000] Q Loss: 0.6927815079689026 
[TRAIN 23000] Q Loss: 0.864874839782714

In [64]:
import torch

In [65]:
env = ENVS_REPLAY_BUFFER[1][1]
model = TaxiNet(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

TaxiNet(
  (fc1): Linear(in_features=500, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=6, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [66]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -7798.28


In [67]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

## Cliffwalking

In [34]:
from util.network import CliffWalkingNet

In [35]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 200_000
GAMMA = 0.98  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = 256
# Extra:

In [36]:
ENVS_REPLAY_BUFFER[2][2].batch_size = BATCH_SIZE

In [37]:
config = get_config(ENVS_REPLAY_BUFFER[2][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [38]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[2][2], CliffWalkingNet)

[TRAIN 0] Q Loss: 2.0012245178222656 
[TRAIN 1000] Q Loss: 0.018101351335644722 
[TRAIN 2000] Q Loss: 0.006127160973846912 
[TRAIN 3000] Q Loss: 0.013781359419226646 
[TRAIN 4000] Q Loss: 0.009676599875092506 
[TRAIN 5000] Q Loss: 0.0005084751755930483 
[TRAIN 6000] Q Loss: 0.001383662805892527 
[TRAIN 7000] Q Loss: 0.0024415438529103994 
[TRAIN 8000] Q Loss: 0.008482544682919979 
[TRAIN 9000] Q Loss: 0.0027746311388909817 
[TRAIN 10000] Q Loss: 0.0008315927698276937 
[TRAIN 11000] Q Loss: 0.0007645394653081894 
[TRAIN 12000] Q Loss: 0.0039224219508469105 
[TRAIN 13000] Q Loss: 0.008235163986682892 
[TRAIN 14000] Q Loss: 0.0007088029524311423 
[TRAIN 15000] Q Loss: 0.0008764057420194149 
[TRAIN 16000] Q Loss: 0.0014097047969698906 
[TRAIN 17000] Q Loss: 0.0018459402490407228 
[TRAIN 18000] Q Loss: 0.0018906047334894538 
[TRAIN 19000] Q Loss: 0.00291275093331933 
[TRAIN 20000] Q Loss: 0.008309327997267246 
[TRAIN 21000] Q Loss: 0.00035604380536824465 
[TRAIN 22000] Q Loss: 0.00227491185

In [39]:
import torch

In [40]:
env = ENVS_REPLAY_BUFFER[2][1]
model = CliffWalkingNet(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[2][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

CliffWalkingNet(
  (fc1): Linear(in_features=48, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=4, bias=True)
)

In [41]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -13.0


In [42]:
extract_and_display(model, ENVS_REPLAY_BUFFER[2][0])

d:\Luiz\Faculdade\RL\OffRL\venv\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Luiz\Faculdade\RL\OffRL\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
